In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip
import astropy.units as u
from astropy.table import Table
import numpy as np
import os
import pandas as pd
from glob import glob

from split_image import bin_image
from profound_estimate_sky import calculate_sky_profound
from make_diagnostic import make_plots

In [2]:
# Read latest zeropoint corrections
# https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-performance/nircam-absolute-flux-calibration-and-zeropoints
best_zp_df = Table.read('/Users/rosaliaobrien/pearls/sky_measurements/NRC_ZPs_1126pmap.txt', 
           format = 'ascii', data_start = 1, guess = False, delimiter = '|',
           names = ['na1', 'pupil+filter', 'detector', 'PHOTMJSR', 'zp_vega', 'vega_Jy', 'zp_vega-sirius', 'vega-sirius_Jy',
                    'zp_AB', 'mean_pix_sr', 'na2']) 

# Add a column with just the filter names
best_zp_df['filter'] = [name.split('+')[1] for name in best_zp_df['pupil+filter']]

# Define function to get ratio of new zeropoint / old zeropoint
def sky_zp_corr(new_ZP, photmjsr):
    
    new_ZP = new_ZP*u.MJy/u.sr
    
    ratio = new_ZP/photmjsr
    
    return ratio.value

In [3]:
# Read in info about the profound images
cal_info_tab = pd.read_csv('/Volumes/treasure/pearls/nep_images/nep_cal_info.csv')
cal_info_tab['root'] = cal_info_tab['file'].str.split('_cal', expand = True)[0]
cal_info_tab

,full,file,stub,path,ext,PHOTMJSR,PIXAR_SR,CRVAL1,CRVAL2,CD1_1,...,ext.1,VISIT_ID,DATE-OBS,DETECTOR,FILTER,EFFEXPTM,CRDS_CTX,MAGZERO,MAGZERO_FIX,root
0,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738001001_02101_00001_nrca1_cal_Pro1oF_sky...,jw02738001001_02101_00001_nrca1_cal_Pro1oF_sky...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,2.251200,2.243274e-14,260.528375,65.823114,0.000008,...,1,2738001001,2023-05-30,NRCA1,F150W,418.734,jwst_1100.pmap,28.022794,27.984042,jw02738001001_02101_00001_nrca1
1,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738001001_02101_00001_nrca2_cal_Pro1oF_sky...,jw02738001001_02101_00001_nrca2_cal_Pro1oF_sky...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,2.383600,2.230000e-14,260.516395,65.804954,0.000008,...,1,2738001001,2023-05-30,NRCA2,F150W,418.734,jwst_1100.pmap,28.029238,28.016378,jw02738001001_02101_00001_nrca2
2,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738001001_02101_00001_nrca3_cal_Pro1oF_sky...,jw02738001001_02101_00001_nrca3_cal_Pro1oF_sky...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,2.203700,2.264701e-14,260.573225,65.818018,0.000008,...,1,2738001001,2023-05-30,NRCA3,F150W,418.734,jwst_1100.pmap,28.012473,27.967924,jw02738001001_02101_00001_nrca3
3,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738001001_02101_00001_nrca4_cal_Pro1oF_sky...,jw02738001001_02101_00001_nrca4_cal_Pro1oF_sky...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,2.102100,2.250000e-14,260.560344,65.799874,0.000008,...,1,2738001001,2023-05-30,NRCA4,F150W,418.734,jwst_1100.pmap,28.019544,28.001666,jw02738001001_02101_00001_nrca4
4,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738001001_02101_00001_nrcalong_cal_Pro1oF_...,jw02738001001_02101_00001_nrcalong_cal_Pro1oF_...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,0.404700,9.209440e-14,260.544667,65.811390,0.000017,...,1,2738001001,2023-05-30,NRCALONG,F356W,418.734,jwst_1100.pmap,26.489417,26.446545,jw02738001001_02101_00001_nrcalong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738008001_12101_00004_nrcblong_cal_Pro1oF_...,jw02738008001_12101_00004_nrcblong_cal_Pro1oF_...,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,0.417500,9.237398e-14,260.727825,65.775531,0.000001,...,1,2738008001,2022-09-14,NRCBLONG,F277W,418.734,jwst_1100.pmap,26.486126,26.479644,jw02738008001_12101_00004_nrcblong
2048,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738008001_12201_00001_nis_cal_Pro1oF_sky_r...,jw02738008001_12201_00001_nis_cal_Pro1oF_sky_rem,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,0.264704,1.009185e-13,260.599139,65.837605,0.000001,...,1,2738008001,2022-09-14,NIS,CLEAR,386.524,jwst_1100.pmap,26.390073,26.390073,jw02738008001_12201_00001_nis
2049,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738008001_12201_00002_nis_cal_Pro1oF_sky_r...,jw02738008001_12201_00002_nis_cal_Pro1oF_sky_rem,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,0.264704,1.009185e-13,260.595089,65.837786,0.000001,...,1,2738008001,2022-09-14,NIS,CLEAR,386.524,jwst_1100.pmap,26.390073,26.390073,jw02738008001_12201_00002_nis
2050,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm/jw02...,jw02738008001_12201_00003_nis_cal_Pro1oF_sky_r...,jw02738008001_12201_00003_nis_cal_Pro1oF_sky_rem,/Volumes/RAIDY/JWST/Pro1oF/cal_sky_renorm,2,0.264704,1.009185e-13,260.594647,65.836128,0.000001,...,1,2738008001,2022-09-14,NIS,CLEAR,386.524,jwst_1100.pmap,26.390073,26.390073,jw02738008001_12201_00003_nis


In [4]:
# Make function to get certain keywords from the above table
key_list = ['CRVAL1', 'CRVAL2', 'PHOTMJSR', 'DATE-OBS', 'VISIT_ID', 'DETECTOR', 'FILTER', 'EFFEXPTM']

def get_header_info(file_root):
    
    dic = {}
    for key in key_list:

        row = cal_info_tab[cal_info_tab['root'] == file_root]

        dic[key] = [row[key].item()]

    return dic

In [5]:
# Get files you will use for sky measurements
file_list = glob('/Volumes/treasure/pearls/nep_images/NEPTDF_sky_files/*.fits')
len(file_list)

df = pd.DataFrame([])

for i, file in enumerate(file_list):

    basefile = os.path.basename(file) # file name
#     fullroot = basefile.split('.')[0] # root name with the _cal
    root = basefile.split('.')[0].split('_cal')[0]

    # Make simple dataframe with file name and root name
    file_df = pd.DataFrame({'file': [basefile], 'root': [root]})
    
    # Get information from headers
#     file_root = os.path.basename(file)[:25]
    phdr_dic = get_header_info(root)
    
    print(i, root)
    
    # Update sci data with updated zeropoints
    flter = phdr_dic['FILTER']
    detector = phdr_dic['DETECTOR']
    
    print(detector, flter)

    # Get data
    hdu = fits.open(file)

    # The profound corrected images have the sky subtracted, 
    # so you need to add the sky back before doing sky measurements
    sky_data = hdu['SKY'].data
    rms_data = hdu['SKYRMS'].data
    
    # Grab new zeropoint
#     new_ZP = best_zp_df[ (best_zp_df['filter'] == flter) & 
#                          (best_zp_df['detector'] == detector)]['PHOTMJSR'].value[0]
    
    # Adjust sky and rms maps to match new zeropoint
#     ratio = sky_zp_corr(new_ZP = new_ZP, photmjsr = phdr_dic['PHOTMJSR'][0]*u.MJy/u.sr)
#     sky_data = sky_data_og*ratio
#     rms_data = rms_data_og*ratio

    # Calculate sky level
    sky_dic = calculate_sky_profound(sky_data = sky_data, rms_data = rms_data, bin_size = 64, 
                                     dq_data = hdu['PIXELMASK'].data, dq_good_list = [0], has_DQ = True, 
                                     dq_fraction = 0.2, percentile = 50)

    hdu.close()

    # Get keywords from headers and stuff
    phdr_df = pd.DataFrame.from_dict(phdr_dic)
    sky_df = pd.DataFrame.from_dict(sky_dic)

    sky_df_forcsv = sky_df[['calc_sky', 'calc_rms', 'N_total_regions', 'N_bad_regions',
                           'N_bad_px_regions', 'N_good_regions']]

    final_df = pd.concat([file_df, sky_df_forcsv, phdr_df], axis = 1)
    df = df.append(final_df)
    df.to_csv('~/pearls/sky_measurements/pearls_neptdf.csv', index = False)

    make_plots(sky_data, sky_df['cutouts'].to_list()[0], sky_df['lowest5perc_ind'].to_list()[0], sky_df['bad_ind'].to_list()[0], badpx = sky_df['badpx_ind'].to_list()[0], 
               sky = sky_df['calc_sky'].item(), rms = sky_df['calc_rms'].item(), show = False, targname = '', 
               save = True, savepath = '../inspection_plots_nep/{r}_prof_insplot.pdf'.format(r = root), 
               title = root)

0 jw02738004002_10101_00001_nrcblong
['NRCBLONG'] ['F410M']
1 jw02738004001_08101_00001_nrcalong
['NRCALONG'] ['F277W']
2 jw02738001001_06101_00001_nrcb1
['NRCB1'] ['F150W']
3 jw02738001001_08101_00003_nrca3
['NRCA3'] ['F200W']
4 jw02738004002_08101_00004_nrcb1
['NRCB1'] ['F200W']
5 jw02738008001_02101_00004_nrcblong
['NRCBLONG'] ['F356W']
6 jw02738003001_12101_00003_nrcb4
['NRCB4'] ['F200W']
7 jw02738002001_04101_00003_nrca4
['NRCA4'] ['F090W']
8 jw02738001002_12101_00003_nrcb4
['NRCB4'] ['F200W']
9 jw02738002001_02201_00004_nis
['NIS'] ['CLEAR']
10 jw02738004001_04101_00003_nrcb4
['NRCB4'] ['F090W']
11 jw02738003002_08101_00003_nrca3
['NRCA3'] ['F200W']
12 jw02738003002_06101_00001_nrcb1
['NRCB1'] ['F150W']
13 jw02738003002_06101_00004_nrca2
['NRCA2'] ['F150W']
14 jw02738003002_10101_00004_nrcblong
['NRCBLONG'] ['F410M']
15 jw02738003001_08101_00004_nrcalong
['NRCALONG'] ['F277W']
16 jw02738001001_04101_00002_nrcalong
['NRCALONG'] ['F444W']
17 jw02738001001_10101_00001_nrca4
['NRCA4'

146 jw02738001002_06101_00002_nrca2
['NRCA2'] ['F150W']
147 jw02738001001_08101_00001_nrcb3
['NRCB3'] ['F200W']
148 jw02738001002_02101_00004_nrcb4
['NRCB4'] ['F150W']
149 jw02738003001_12101_00001_nrca4
['NRCA4'] ['F200W']
150 jw02738004002_06101_00004_nrcb3
['NRCB3'] ['F150W']
151 jw02738004001_08101_00002_nrcb1
['NRCB1'] ['F200W']
152 jw02738008001_12101_00001_nrcb4
['NRCB4'] ['F200W']
153 jw02738004001_12101_00002_nrcalong
['NRCALONG'] ['F277W']
154 jw02738004002_08101_00003_nrcb2
['NRCB2'] ['F200W']
155 jw02738003002_10101_00003_nrcb4
['NRCB4'] ['F115W']
156 jw02738004001_04101_00004_nrcblong
['NRCBLONG'] ['F444W']
157 jw02738008001_08101_00003_nrcblong
['NRCBLONG'] ['F277W']
158 jw02738002001_08101_00002_nrca1
['NRCA1'] ['F200W']
159 jw02738008001_06101_00002_nrcb2
['NRCB2'] ['F150W']
160 jw02738001001_10101_00003_nrcb4
['NRCB4'] ['F115W']
161 jw02738008001_08201_00004_nis
['NIS'] ['CLEAR']
162 jw02738008001_02101_00004_nrca4
['NRCA4'] ['F150W']
163 jw02738001001_06101_00004_nrcb

290 jw02738002001_06101_00003_nrca2
['NRCA2'] ['F150W']
291 jw02738002001_08201_00002_nis
['NIS'] ['CLEAR']
292 jw02738004002_02101_00002_nrcalong
['NRCALONG'] ['F356W']
293 jw02738002001_02201_00003_nis
['NIS'] ['CLEAR']
294 jw02738004001_06101_00001_nrcblong
['NRCBLONG'] ['F356W']
295 jw02738008001_06101_00001_nrca3
['NRCA3'] ['F150W']
296 jw02738008001_08101_00003_nrcb1
['NRCB1'] ['F200W']
297 jw02738004002_10101_00002_nrca4
['NRCA4'] ['F115W']
298 jw02738001001_04101_00004_nrca4
['NRCA4'] ['F090W']
299 jw02738003001_08101_00003_nrca1
['NRCA1'] ['F200W']
300 jw02738004002_12101_00004_nrca4
['NRCA4'] ['F200W']
301 jw02738003002_08101_00002_nrca2
['NRCA2'] ['F200W']
302 jw02738003001_06101_00001_nrcb3
['NRCB3'] ['F150W']
303 jw02738002001_08101_00004_nrca3
['NRCA3'] ['F200W']
304 jw02738001001_08101_00002_nrca2
['NRCA2'] ['F200W']
305 jw02738001002_06101_00001_nrcb3
['NRCB3'] ['F150W']
306 jw02738003001_06101_00004_nrcblong
['NRCBLONG'] ['F356W']
307 jw02738004002_02101_00001_nrcb4
['

435 jw02738002001_06101_00004_nrca1
['NRCA1'] ['F150W']
436 jw02738001002_08101_00001_nrcb1
['NRCB1'] ['F200W']
437 jw02738008001_04101_00002_nrcb4
['NRCB4'] ['F090W']
438 jw02738004001_06101_00004_nrcb1
['NRCB1'] ['F150W']
439 jw02738003002_06201_00001_nis
['NIS'] ['CLEAR']
440 jw02738003001_08101_00001_nrcb1
['NRCB1'] ['F200W']
441 jw02738008001_08101_00004_nrcb2
['NRCB2'] ['F200W']
442 jw02738008001_06101_00003_nrcalong
['NRCALONG'] ['F356W']
443 jw02738008001_08201_00003_nis
['NIS'] ['CLEAR']
444 jw02738004001_02201_00001_nis
['NIS'] ['CLEAR']
445 jw02738003001_06101_00003_nrca3
['NRCA3'] ['F150W']
446 jw02738003001_06101_00001_nrcalong
['NRCALONG'] ['F356W']
447 jw02738008001_10101_00002_nrcb3
['NRCB3'] ['F115W']
448 jw02738003002_02101_00002_nrcblong
['NRCBLONG'] ['F356W']
449 jw02738004001_02101_00003_nrcb2
['NRCB2'] ['F150W']
450 jw02738004002_02101_00002_nrcb1
['NRCB1'] ['F150W']
451 jw02738002001_02101_00003_nrca2
['NRCA2'] ['F150W']
452 jw02738008001_04101_00003_nrca1
['NRCA

579 jw02738003001_10101_00003_nrcb3
['NRCB3'] ['F115W']
580 jw02738003002_12201_00003_nis
['NIS'] ['CLEAR']
581 jw02738004002_02101_00003_nrca1
['NRCA1'] ['F150W']
582 jw02738001002_02101_00003_nrcblong
['NRCBLONG'] ['F356W']
583 jw02738002001_02101_00002_nrcalong
['NRCALONG'] ['F356W']
584 jw02738002001_06101_00004_nrca4
['NRCA4'] ['F150W']
585 jw02738008001_10101_00003_nrca3
['NRCA3'] ['F115W']
586 jw02738001002_08101_00001_nrcb4
['NRCB4'] ['F200W']
587 jw02738004001_02101_00002_nrca2
['NRCA2'] ['F150W']
588 jw02738003001_06201_00002_nis
['NIS'] ['CLEAR']
589 jw02738001002_04101_00001_nrcalong
['NRCALONG'] ['F444W']
590 jw02738004002_02201_00002_nis
['NIS'] ['CLEAR']
591 jw02738001001_02101_00004_nrcb3
['NRCB3'] ['F150W']
592 jw02738003002_12101_00001_nrca3
['NRCA3'] ['F200W']
593 jw02738002001_02101_00002_nrcb2
['NRCB2'] ['F150W']
594 jw02738008001_04101_00002_nrcb1
['NRCB1'] ['F090W']
595 jw02738004001_06101_00004_nrcb4
['NRCB4'] ['F150W']
596 jw02738003002_02101_00004_nrcb3
['NRCB

...

1439 jw02738004002_04101_00001_nrca2
['NRCA2'] ['F090W']
1440 jw02738003002_06101_00002_nrca4
['NRCA4'] ['F150W']
1441 jw02738003002_08201_00001_nis
['NIS'] ['CLEAR']
1442 jw02738002001_10101_00001_nrca3
['NRCA3'] ['F115W']
1443 jw02738008001_06201_00003_nis
['NIS'] ['CLEAR']
1444 jw02738003002_02101_00004_nrcb2
['NRCB2'] ['F150W']
1445 jw02738001001_12101_00001_nrca2
['NRCA2'] ['F200W']
1446 jw02738004001_10101_00001_nrcb3
['NRCB3'] ['F115W']
1447 jw02738001001_02101_00004_nrcb2
['NRCB2'] ['F150W']
1448 jw02738002001_02101_00002_nrcb3
['NRCB3'] ['F150W']
1449 jw02738003002_12101_00001_nrca2
['NRCA2'] ['F200W']
1450 jw02738008001_06101_00001_nrcalong
['NRCALONG'] ['F356W']
1451 jw02738008001_10101_00003_nrca2
['NRCA2'] ['F115W']
1452 jw02738001001_06101_00002_nrca4
['NRCA4'] ['F150W']
1453 jw02738004001_02101_00002_nrca3
['NRCA3'] ['F150W']
1454 jw02738002001_04101_00002_nrcb2
['NRCB2'] ['F090W']
1455 jw02738001002_12101_00002_nrca2
['NRCA2'] ['F200W']
1456 jw02738001001_04101_00004_nr

1581 jw02738001001_02101_00002_nrcb2
['NRCB2'] ['F150W']
1582 jw02738003001_04101_00001_nrcb3
['NRCB3'] ['F090W']
1583 jw02738002001_02101_00004_nrcb3
['NRCB3'] ['F150W']
1584 jw02738001002_02101_00003_nrcb1
['NRCB1'] ['F150W']
1585 jw02738001001_04101_00001_nrcalong
['NRCALONG'] ['F444W']
1586 jw02738008001_12101_00003_nrca2
['NRCA2'] ['F200W']
1587 jw02738004001_02101_00004_nrca3
['NRCA3'] ['F150W']
1588 jw02738001001_06101_00004_nrca4
['NRCA4'] ['F150W']
1589 jw02738001001_02101_00003_nrcblong
['NRCBLONG'] ['F356W']
1590 jw02738003002_10101_00001_nrca2
['NRCA2'] ['F115W']
1591 jw02738002001_06101_00001_nrcblong
['NRCBLONG'] ['F356W']
1592 jw02738001001_10101_00001_nrca2
['NRCA2'] ['F115W']
1593 jw02738003002_06101_00004_nrca4
['NRCA4'] ['F150W']
1594 jw02738004001_12101_00001_nrcb3
['NRCB3'] ['F200W']
1595 jw02738003001_02101_00003_nrcb1
['NRCB1'] ['F150W']
1596 jw02738003002_02101_00002_nrcb2
['NRCB2'] ['F150W']
1597 jw02738002001_12101_00001_nrca3
['NRCA3'] ['F200W']
1598 jw027380

1724 jw02738002001_12101_00003_nrcb3
['NRCB3'] ['F200W']
1725 jw02738001002_04101_00003_nrca3
['NRCA3'] ['F090W']
1726 jw02738004002_08101_00003_nrcb4
['NRCB4'] ['F200W']
1727 jw02738003001_04101_00003_nrca3
['NRCA3'] ['F090W']
1728 jw02738001002_02101_00001_nrca1
['NRCA1'] ['F150W']
1729 jw02738003001_12101_00004_nrcb1
['NRCB1'] ['F200W']
1730 jw02738008001_12101_00001_nrcb2
['NRCB2'] ['F200W']
1731 jw02738003001_10101_00002_nrcb1
['NRCB1'] ['F115W']
1732 jw02738003002_10101_00003_nrcb2
['NRCB2'] ['F115W']
1733 jw02738003002_06101_00003_nrcblong
['NRCBLONG'] ['F356W']
1734 jw02738004002_02101_00002_nrca3
['NRCA3'] ['F150W']
1735 jw02738001002_06101_00002_nrca4
['NRCA4'] ['F150W']
1736 jw02738003001_12101_00004_nrcalong
['NRCALONG'] ['F277W']
1737 jw02738008001_10101_00002_nrca1
['NRCA1'] ['F115W']
1738 jw02738008001_12101_00004_nrca1
['NRCA1'] ['F200W']
1739 jw02738003001_12101_00001_nrca2
['NRCA2'] ['F200W']
1740 jw02738003001_04101_00002_nrcblong
['NRCBLONG'] ['F444W']
1741 jw027380

1865 jw02738001001_04101_00002_nrcblong
['NRCBLONG'] ['F444W']
1866 jw02738008001_10101_00003_nrcb3
['NRCB3'] ['F115W']
1867 jw02738003001_08101_00004_nrcblong
['NRCBLONG'] ['F277W']
1868 jw02738001002_08101_00001_nrca4
['NRCA4'] ['F200W']
1869 jw02738003002_10101_00004_nrcalong
['NRCALONG'] ['F410M']
1870 jw02738002001_06101_00004_nrcb4
['NRCB4'] ['F150W']
1871 jw02738004001_02101_00002_nrcb2
['NRCB2'] ['F150W']
1872 jw02738004002_04101_00001_nrcb3
['NRCB3'] ['F090W']
1873 jw02738003001_08101_00001_nrca4
['NRCA4'] ['F200W']
1874 jw02738002001_10101_00001_nrcb2
['NRCB2'] ['F115W']
1875 jw02738001001_12101_00001_nrcb3
['NRCB3'] ['F200W']
1876 jw02738003002_02101_00004_nrca3
['NRCA3'] ['F150W']
1877 jw02738004001_10101_00001_nrca2
['NRCA2'] ['F115W']
1878 jw02738008001_04101_00003_nrcb4
['NRCB4'] ['F090W']
1879 jw02738004002_06101_00004_nrcb2
['NRCB2'] ['F150W']
1880 jw02738001001_08101_00001_nrcb2
['NRCB2'] ['F200W']
1881 jw02738001002_06101_00002_nrca3
['NRCA3'] ['F150W']
1882 jw027380

2007 jw02738001001_12101_00001_nrcblong
['NRCBLONG'] ['F277W']
2008 jw02738001002_08101_00002_nrca1
['NRCA1'] ['F200W']
2009 jw02738001001_08101_00003_nrca2
['NRCA2'] ['F200W']
2010 jw02738003001_08101_00001_nrcalong
['NRCALONG'] ['F277W']
2011 jw02738003002_10101_00001_nrcblong
['NRCBLONG'] ['F410M']
2012 jw02738001001_06101_00004_nrca3
['NRCA3'] ['F150W']
2013 jw02738002001_06101_00002_nrca2
['NRCA2'] ['F150W']
2014 jw02738004001_02101_00004_nrca4
['NRCA4'] ['F150W']
2015 jw02738004002_06101_00003_nrcb1
['NRCB1'] ['F150W']
2016 jw02738002001_02101_00004_nrcb4
['NRCB4'] ['F150W']
2017 jw02738003001_04101_00001_nrcb4
['NRCB4'] ['F090W']
2018 jw02738004001_06101_00002_nrcb2
['NRCB2'] ['F150W']
2019 jw02738004002_08101_00001_nrca3
['NRCA3'] ['F200W']
2020 jw02738001002_04101_00001_nrcb4
['NRCB4'] ['F090W']
2021 jw02738002001_12101_00001_nrca4
['NRCA4'] ['F200W']
2022 jw02738008001_02101_00001_nrcblong
['NRCBLONG'] ['F356W']
2023 jw02738004001_12101_00001_nrcb4
['NRCB4'] ['F200W']
2024 jw

In [6]:
fits.open(file)[3].header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                   32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2048 / length of data axis 1                          
NAXIS2  =                 2048 / length of data axis 2                          
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
EXTNAME = 'PIXELMASK'          / extension name                                 